# #Problem Recap
# # Max-Cut Problem Definition

# ## Graph Definition

# We consider an undirected, unweighted graph  
# \[
# G = (V, E)
# \]
# where:
# - \( V = \{0, 1, 2, 3\} \) is the set of vertices
# - \( E = \{(0,1), (0,2), (1,2), (1,3)\} \) is the set of edges

# Each vertex represents a binary decision variable, and each edge represents a constraint between two variables.

# ---

# ## Max-Cut Problem

# The **Max-Cut problem** asks us to partition the set of vertices \(V\) into two disjoint subsets such that the number of edges connecting vertices in different subsets is maximized.

# ---

# ## Binary Representation of a Cut

# We associate each vertex \(i\) with a binary variable:
# \[
# x_i \in \{0,1\}
# \]

# - \(x_i = 0\) means vertex \(i\) is assigned to subset A  
# - \(x_i = 1\) means vertex \(i\) is assigned to subset B  

# A **cut** is therefore represented by a bitstring:
# \[
# x = (x_0, x_1, x_2, x_3)
# \]

# ---

# ## Max-Cut Cost Function

# For an edge \((i,j) \in E\), the edge contributes **1** to the cut if the two endpoints are in different subsets:
# \[
# [x_i \neq x_j]
# \]

# The total cut value is:
# \[
# C(x) = \sum_{(i,j)\in E} [x_i \neq x_j]
# \]

# ---

# ## Objective

# The goal of Max-Cut is to find the bitstring \(x\) that maximizes \(C(x)\).

# This classical cost function will be transformed into:
# - a QUBO formulation
# - an Ising Hamiltonian
# - a quantum cost operator for QAOA

# # Max-Cut as a QUBO Problem

# To prepare the Max-Cut problem for quantum optimization, we first rewrite the cost function in **algebraic form** using binary variables.

# ---

# ## Binary Variables

# Each vertex \(i \in V\) is assigned a binary variable:
# \[
# x_i \in \{0,1\}
# \]

# A cut is represented by the bitstring:
# \[
# x = (x_0, x_1, x_2, x_3)
# \]

# ---

# ## Edge Contribution

# For an edge \((i,j)\), the edge contributes **1** to the cut if the two vertices belong to different subsets:
# \[
# [x_i \neq x_j]
# \]

# This indicator function can be rewritten algebraically as:
# \[
# [x_i \neq x_j] = x_i + x_j - 2x_i x_j
# \]

# ---

# ## QUBO Cost Function

# Substituting this expression into the Max-Cut objective gives:
# \[
# C(x) = \sum_{(i,j)\in E} \left( x_i + x_j - 2x_i x_j \right)
# \]

# This is a **Quadratic Unconstrained Binary Optimization (QUBO)** problem, since:
# - The variables \(x_i\) are binary
# - The cost function contains linear and quadratic terms
# - There are no explicit constraints

# ---

# ## Optimization Objective

# The goal is to find a binary vector \(x\) that **maximizes** \(C(x)\).

# In the next step, this QUBO formulation will be converted into an Ising model suitable for quantum computation.

# Converting QUBO to an Ising Model

Quantum algorithms such as QAOA operate on **spin variables** rather than binary variables.  
Therefore, we convert the QUBO formulation into an equivalent **Ising model**.

---

## Change of Variables

Each binary variable \(x_i \in \{0,1\}\) is mapped to a spin variable:
\[
z_i \in \{-1, +1\}
\]

The transformation is:
\[
x_i = \frac{1 - z_i}{2}
\]

This mapping ensures a one-to-one correspondence between binary assignments and spin configurations.

---

## Substitution into the Cost Function

Starting from the QUBO form:
\[
C(x) = \sum_{(i,j)\in E} \left( x_i + x_j - 2x_i x_j \right)
\]

Substituting \(x_i = \frac{1 - z_i}{2}\) gives:
\[
C(z) = \sum_{(i,j)\in E} \frac{1}{2}\left(1 - z_i z_j\right)
\]

Constant terms do not affect the optimization objective and can be ignored when constructing the Hamiltonian.

---

## Ising Hamiltonian

Replacing spin variables \(z_i\) with Pauli-Z operators \(Z_i\), we obtain the **cost Hamiltonian**:
\[
H_C = \sum_{(i,j)\in E} \frac{1}{2}\left(1 - Z_i Z_j\right)
\]

---

## Interpretation

- Each term penalizes configurations where connected vertices have the same spin
- The ground state of \(H_C\) corresponds to the maximum cut
- This Hamiltonian will be used directly in the QAOA cost unitary

---

## Next Step

In the next cell, we will implement this Ising cost function in code and verify that it matches the classical Max-Cut values.

In [3]:
import numpy as np

edges = [(0,1), (0,2), (1,2), (1,3)]

def ising_energy(bitstring, edges):
    z = [1 if b == 0 else -1 for b in bitstring]
    energy = 0
    for i, j in edges:
        energy += 0.5 * (1 - z[i] * z[j])
    return energy


In [4]:
from itertools import product

for bits in product([0,1], repeat=4):
    print(bits, ising_energy(bits, edges))

(0, 0, 0, 0) 0.0
(0, 0, 0, 1) 1.0
(0, 0, 1, 0) 2.0
(0, 0, 1, 1) 3.0
(0, 1, 0, 0) 3.0
(0, 1, 0, 1) 2.0
(0, 1, 1, 0) 3.0
(0, 1, 1, 1) 2.0
(1, 0, 0, 0) 2.0
(1, 0, 0, 1) 3.0
(1, 0, 1, 0) 2.0
(1, 0, 1, 1) 3.0
(1, 1, 0, 0) 3.0
(1, 1, 0, 1) 2.0
(1, 1, 1, 0) 1.0
(1, 1, 1, 1) 0.0
